In [29]:
import csv
import os
import pandas as pd
import sacrebleu
import pyter
import numpy as np
def get_length(preds, truth):
    preds_len = get_mean_len(preds)
    truth_len = get_mean_len(truth)
    return preds_len,truth_len
    
def get_mean_len(hyp):
    length = 0
    for row in hyp:
        length+=len(row)
    return length/len(hyp)
    
def get_scores(preds, truth):
    preds_score = get_selfbleu(preds)
    truth_score = get_selfbleu(truth)
    return preds_score, truth_score

def get_selfbleu(hyp):
    self_bleu = 0
    hyp_ = np.array(hyp)
    for idx in range(len(hyp)):
        rev_idx = np.arange(len(hyp))
        # [True, True, True, True, False, True, False, True, True, True] を作れば良いという方針
        bool_idx = np.ones(len(hyp), dtype=bool)
        bool_idx[idx] = False
        rev_idx = rev_idx[bool_idx]
        for rev in rev_idx:
            tmp_score = sacrebleu.sentence_bleu(hyp_[idx], [hyp_[rev]])
            self_bleu += tmp_score.score
    self_bleu /= len(hyp)*(len(hyp)-1)
    return self_bleu

def get_data_as_list(path):
    data = []
    with open(path, 'r', encoding='utf-8-sig')as f:
        reader = csv.reader(f)
        for row in reader:
            data.append(row)
    return data



In [38]:
ver_name = '000_translate_all_both_prefix_rel'
context = 2
lang_list = ['ja','zh']
selfbleu_len = []
for lang in lang_list:
    f_name = f'{lang}_preds_truth.csv'
    path = f'outputs/context/{ver_name}/{context}/{f_name}'
    df = pd.read_csv(path, encoding='utf-8-sig')
    generated_s,truth_s = get_scores(df[ver_name].to_list(),df['truth'].to_list())
    generated_l,truth_l = get_length(df[ver_name].to_list(),df['truth'].to_list())
    selfbleu_len.append([lang,'selfbleu',truth_s,generated_s])
    selfbleu_len.append([lang,'length',generated_l,truth_l])

    
print(selfbleu_len)
os.makedirs('for_thesis/selfbleu_len/',exist_ok=True)
with open(f'for_thesis/selfbleu_len/{ver_name}_{context}.csv', 'w',encoding='utf-8-sig')as f:
    writer = csv.writer(f)
    writer.writerows(selfbleu_len)
    

[['ja', 'selfbleu', 37.85449511476157, 44.13366288819339], ['ja', 'length', 21.8875, 31.50625], ['zh', 'selfbleu', 20.573110022340515, 40.05867202166839], ['zh', 'length', 9.885057471264368, 13.798850574712644]]


20.573110022340515 40.05867202166839
9.885057471264368 13.798850574712644
